# Otras arquitecturas "avanzadas"

Hasta ahora basicamente hemos visto problemas de etiquetar documentos como un tipo de clase bastante definido, y normalmente como "documentos standalone", es decir, que en nuestras arquitecturas, *x* era documento solo, y la clase solía ser, me gusta/no me gusta, política/deporte/música, en fin, etiquetas en general.

En sequence to sequence, vimos arquitecturas que nos permetían generar un output, basándonos en dos inputs, un primer input que condicionaba la salida (encoder), y otro input que era el que queríamos encontrarnos en el output (decoder).

Imaginaros que ahora queremos tener arquitecturas que nos dado pares de frases nos dicen si estas frases más o menos significan lo mismo. O en el caso del primer uso de la arquitectura que implementaremos, si dada una premisa, la hypothesis es correcta. Es decir inferir si dado x, se da y.

A esto se le llama Natural Language Infernece.

<div align="center">
    ![](http://wx3.sinaimg.cn/large/006Fmjmcly1fhiby4jojuj31dk0r6qkz.jpg =600x)
</div>



## Natural Language Inference

[SNLI](https://nlp.stanford.edu/projects/snli/) es una página de stanford que se dedica a trackear el progreso hecho en este ámbito.

La arquitectura, empieza más o menos igual que el sequence to sequence, pero en lugar de encoder-decoder, usaremos 2 encoders (LSTMs) y luego lo moveremos todo para arriba, donde clasificaremos las siguientes 3 clases: 

* Entailment
* Neutral
* Contradiction

<div align="center">
    ![](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/29f34ca5778f742b2cee0dfc70819eca11759208/3-Figure1-1.png =400x)
</div>

Ahora que se esta empezando a luchar contra las fake-news con models de Machine Learning y Deep Learning, esta arquitectura tiene muchísimo sentido, pues por ejemplo podríamos pasar una notícia, y un seguido de "facts" y ver si la relación es de entailment o contradiction, y predecir si es fake-news o no.



## Imports

In [0]:
import numpy as np
from random import uniform, shuffle
from random import sample

## Data Generation

In [0]:
equals = []
for _ in range(1000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 1
    equals.append((x0, x1, np.array([1, 0, 0])))

for _ in range(1000):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 0
    equals.append((x0, x1, np.array([1, 0, 0])))
            

In [0]:
meh = []
for _ in range(2000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.50:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.50:
            x1[ix] = 1
    meh.append((x0, x1, np.array([0, 1, 0])))


In [0]:
different = []
for _ in range(1000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.80:
            x1[ix] = 1
    different.append((x0, x1,  np.array([0, 0, 1])))

for _ in range(1000):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.8:
            x1[ix] = 0
    different.append((x0, x1, np.array([0, 0, 1])))

## Model Definition

<div align="center">
    ![](https://i.imgur.com/g8jzlfx.png =450x)
</div>


In [0]:
from keras.layers import Dense, CuDNNLSTM, Dropout, Embedding, Input, Concatenate, Average, Multiply, Add, LSTM
from keras.models import Model
import keras.backend as K

## Dataset Preparation

In [0]:
dataset = equals+meh+different
shuffle(dataset)

## Fit model

In [0]:
for eq in sample(dataset, k=3):
    print(np.sum(eq[0].reshape(1,-1)), np.sum(eq[1].reshape(1,-1)))
    print(model.predict(x=[eq[0].reshape(1,-1), eq[1].reshape(1,-1)]).round())

75.0 79.0
[[1. 0. 0.]]
74.0 23.0
[[0. 0. 1.]]
66.0 82.0
[[1. 0. 0.]]


## Evaluate Model

In [0]:
test_set = []
for _ in range(100):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 1
    test_set.append((x0, x1, np.array([1, 0, 0])))

for _ in range(100):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 0
    test_set.append((x0, x1, np.array([1, 0, 0])))

for _ in range(200):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.50:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.50:
            x1[ix] = 1
    test_set.append((x0, x1, np.array([0, 1, 0])))
    
for _ in range(100):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.80:
            x1[ix] = 1
    test_set.append((x0, x1,  np.array([0, 0, 1])))

for _ in range(100):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.8:
            x1[ix] = 0
    test_set.append((x0, x1, np.array([0, 0, 1])))
    



((600, 100), (600, 100), (600, 3))